In [ ]:
import PIL.Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation
matplotlib.rcParams['animation.html'] = 'html5'

In [ ]:
def load_mask(path):
    with PIL.Image.open(path) as image:
        pixels = np.array(image)
        assert pixels.ndim == 2
        return pixels < 128

mask = load_mask('../app/src/main/assets/two_square128.png')
# mask = load_mask('../app/src/main/assets/two_square128.png')
# plt.imshow(~mask, cmap='gray')
# plt.axis('off')

In [ ]:
def wave_image(x, mask):
    x = np.clip(x, -1, 1)
#     if not ((-1 <= x) & (x <= 1)).all():
#         raise ValueError('State out-of-bounds - should be (-1, 1)-bounded')
    shade = .5 * (1 + x)
    return np.where(mask[..., np.newaxis], np.array([[[1., 0, 0]]]), np.tile(shade[..., np.newaxis], (1, 1, 3)))

def simulate(x0, dx0, mask):
    steps = 100
    inner_steps = 100
    dt = .01
    tension = 10
    damping = 0.0001
    x, dx = x0.copy(), dx0.copy()
    for _ in range(steps):
        for _ in range(inner_steps):
            xpad = np.pad(x, 1, mode='edge')
            x += .5 * dt * dx
            x *= ~mask
            f = .25 * (xpad[:-2, 1:-1] + xpad[2:, 1:-1] + xpad[1:-1, :-2] + xpad[1:-1, 2:]) - x
            dx *= (1 - damping)
            dx += f * tension * dt
            x += .5 * dt * dx
            x *= ~mask
#         x = np.clip(x, -1, 1)
        yield x, dx


# Initial conditions
x0, dx0 = np.zeros(mask.shape), np.zeros(mask.shape)
d = mask.shape[1]//4
b = 4
x0[d-b:d+b, d-b:d+b] = 1
x0[3*d-b:3*d+b, 3*d-b:3*d+b] = -1

# Simulation
states = simulate(x0, dx0, mask)

# Rendering
fig = plt.figure(figsize=(8, 8))
im = plt.imshow(wave_image(x0, mask), animated=True)
plt.axis('off')
plt.close()
def updatefig(state):
    x, dx = state
    im.set_array(wave_image(x, mask))
    return im,
matplotlib.animation.FuncAnimation(fig, updatefig, states, interval=100, save_count=100, blit=True)